In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 700)
import csv
import json

In [ ]:
with open('json/dept_lst.txt', 'r', encoding='utf8') as filehandle:
    dept_lst = json.load(filehandle)

In [ ]:
# append AD.csv
li = []
for str_dept in dept_lst:
    filepath = r"AD_csv\\"+str_dept+'.csv'    
    df = pd.read_csv(filepath, index_col=None, header=0)
    df['OrganizationalUnit'] = str_dept
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

df_AD = frame[frame.title.notnull()].copy()

In [ ]:
# df_CK = df_AD[df_AD['description'].notnull()]
# df_CK[df_CK['description'].str.contains('Leave without pay')]

In [ ]:
df_AD.rename(
    columns = {'department': 'dept_full',
               'OrganizationalUnit': 'dept',
               'displayname': 'name',
               'officephone': 'tel',
              }, inplace = True)

In [ ]:
with open('json/dic_ppl.txt', 'r', encoding='utf8') as filehandle:
    dic_ppl = json.load(filehandle)

In [ ]:
df_AD

In [ ]:
# 董事長特助    
s1=df_AD[df_AD['name'] ==dic_ppl['ChairmanSA']['name']].iloc[0]
d1=s1.to_dict()
d1['dept']=dic_ppl['ChairmanSA']['dept']
d1['dept_full']=dic_ppl['ChairmanSA']['dept_full']
d1['title']=dic_ppl['ChairmanSA']['title']

df_AD.drop(df_AD.loc[df_AD['name']==dic_ppl['ChairmanSA']['name']].index, inplace=True)

df_AD=df_AD.append(d1 , ignore_index=True)

In [ ]:
# 總經理特助 
s2=df_AD[df_AD['name'] ==dic_ppl['PresidentSA']['name']].iloc[0]
d2=s2.to_dict()
d2['dept']=dic_ppl['PresidentSA']['dept']
d2['dept_full']=dic_ppl['PresidentSA']['dept_full']
d2['title']=dic_ppl['PresidentSA']['title']

df_AD.drop(df_AD.loc[df_AD['name']==dic_ppl['PresidentSA']['name']].index, inplace=True)

df_AD=df_AD.append(d2 , ignore_index=True)

In [ ]:
# 兼任OCC長官
socc1=df_AD[df_AD['name'] ==dic_ppl['OCCdirector']['name']].iloc[0]
docc1=socc1.to_dict()
docc1['dept']=dic_ppl['OCCdirector']['dept']
docc1['dept_full']=dic_ppl['OCCdirector']['dept_full']
docc1['title']=dic_ppl['OCCdirector']['title']
df_AD=df_AD.append(docc1, ignore_index=True)

socc2=df_AD[df_AD['name'] ==dic_ppl['OCCviceDirector']['name']].iloc[0]
docc2=socc2.to_dict()
docc2['dept']=dic_ppl['OCCviceDirector']['dept']
docc2['dept_full']=dic_ppl['OCCviceDirector']['dept_full']
docc2['title']=dic_ppl['OCCviceDirector']['title']
df_AD=df_AD.append(docc2, ignore_index=True)

In [ ]:
# 科別順序
with open('json/order_of_section.txt', 'r', encoding='utf8') as filehandle:
    section_order = json.load(filehandle)
df_section_ord = pd.DataFrame(section_order, columns =['section_order', 'section_name'])

In [ ]:
df_AD = pd.merge(df_AD, df_section_ord, left_on='dept_full', right_on='section_name', how='left')
df_AD = df_AD.drop(['section_name'], axis=1)

In [ ]:
# 職稱順序
with open('json/order_of_title.txt', 'r', encoding='utf8') as filehandle:
    title_order = json.load(filehandle)
df_title_ord = pd.DataFrame(title_order, columns =['title_name', 'title_order'])

In [ ]:
df_AD = pd.merge(df_AD, df_title_ord, left_on='title', right_on='title_name', how='left')
df_AD = df_AD.drop(['title_name'], axis=1)

In [ ]:
df_AD = df_AD.sort_values(['section_order', 'title_order', 'tel'],
                  ascending=[True, False, True])

In [ ]:
# 管理處 無科別
df_AD['dept_full'] = np.where(df_AD['dept'] == '總管理處', '總管理處', df_AD['dept_full'])

In [ ]:
# 航務處 消防大隊
df_AD['dept_full'] = np.where(df_AD['dept'] == '消防大隊', '航務處-消防大隊', df_AD['dept_full'])
df_AD['dept'] = np.where(df_AD['dept'] == '消防大隊', '航務處', df_AD['dept'])

In [ ]:
df_AD  = df_AD[[
#                 'samaccountname',
                'dept','dept_full','title','name','tel','mail',
#                 'description',
#                 'section_order', 'title_order'
              ]]

In [ ]:
# 投票使用
# df_AD  = df_AD[[            
#                 'dept','dept_full',
#                 'samaccountname',
#                 'name','mail',
#               ]]
# df_AD['dept_full'] = df_AD['dept_full'].str.replace('-','/')

In [ ]:
df_AD.to_excel('emp_directory_SAMPLE.xlsx', index=False) 

### 準備轉換成遞交格式

In [ ]:
# df_full_cols = pd.read_csv('315850000M_原承整理提交版本.csv',header=None)
idx_col = [
    (0, 'dept_full'),
    (2, 'name'),
    (4, 'mail'),
    (44, 'company_name'),
    (50, 'tel_full'),
    (65, 'dept'),
    (66, 'title'),
]
idx_lst = [tpl[0] for tpl in idx_col]
col_lst = [tpl[1] for tpl in idx_col]

In [ ]:
df_AD['dept_full'] = df_AD['dept_full'].str.replace('-','/')
df_AD['dept_full'] = '/' + df_AD['dept_full'] + '/'

In [ ]:
# 補齊tel欄位
df_AD['tel'].replace(np.nan, 0, inplace=True)
df_AD['tel_full'] = df_AD['tel'].astype(np.int64)
df_AD['tel_full'] = '(03)27' + df_AD['tel_full'].astype(str)
df_AD.drop(columns=['tel',], inplace=True)

In [ ]:
with open('json/company_name.txt', 'r', encoding='utf8') as filehandle:
    company_name_lst = json.load(filehandle)
df_AD['company_name'] = company_name_lst[0]

In [ ]:
# 產生 renew_idx_col
pop_lst = col_lst.copy()

renew_idx_col=[]
for i in range(67):
    if i in idx_lst:
        v = pop_lst.pop(0)
        renew_idx_col.append((i,v))
    else:
        renew_idx_col.append((i,''))

In [ ]:
for t in renew_idx_col:
    if t[1]=='':
        df_AD[t[0]]=''

In [ ]:
col_order_lst = []
for t in renew_idx_col:
    if t[1] != '':
        col_order_lst.append(t[1])
    else:
        col_order_lst.append(t[0])

In [ ]:
# 輸出csv
df_AD = df_AD[col_order_lst]
df_AD.to_csv('emp_directory.csv',index=False, header=False, quoting=csv.QUOTE_ALL)